In [487]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [488]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [489]:
alphavantage_csv_path = "/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL.csv"

In [490]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date_time"] = pd.to_datetime(df['date'])

df["day_of_week"] = df["date_time"].dt.dayofweek

df["adjusted_close_shift"] = df["5. adjusted close"].shift(-1)

df["percentage_change"] = (df["adjusted_close_shift"] - df["5. adjusted close"]) / df["5. adjusted close"] * 100.0

df["increase"] = df["percentage_change"].apply(lambda x: 1 if x > 0 else 0)

df.set_index("date", inplace=True)

In [491]:
df.shape[0]

3064

In [492]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   1. open               3064 non-null   float64       
 1   2. high               3064 non-null   float64       
 2   3. low                3064 non-null   float64       
 3   4. close              3064 non-null   float64       
 4   5. adjusted close     3064 non-null   float64       
 5   6. volume             3064 non-null   float64       
 6   7. dividend amount    3064 non-null   float64       
 7   8. split coefficient  3064 non-null   float64       
 8   date_time             3064 non-null   datetime64[ns]
 9   day_of_week           3064 non-null   int64         
 10  adjusted_close_shift  3063 non-null   float64       
 11  percentage_change     3063 non-null   float64       
 12  increase              3064 non-null   int64         
dtypes: datet

In [493]:
df.tail()
# df[df.percentage_change == 0]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase
date,,,,,,,,,,,,,
2022-02-28,163.060,165.42,162.43,165.12,165.12,95056629.0,0.0,1.0,2022-02-28,0,163.20,-1.162791,0
2022-03-01,164.695,166.60,161.97,163.20,163.20,83474425.0,0.0,1.0,2022-03-01,1,166.56,2.058824,1
2022-03-02,164.390,167.36,162.95,166.56,166.56,79724750.0,0.0,1.0,2022-03-02,2,166.23,-0.198127,0
2022-03-03,168.470,168.91,165.55,166.23,166.23,76678441.0,0.0,1.0,2022-03-03,3,163.17,-1.840823,0
2022-03-04,164.490,165.55,162.10,163.17,163.17,83819592.0,0.0,1.0,2022-03-04,4,NaN,NaN,0


In [494]:
df.increase.value_counts()

1    1621
0    1443
Name: increase, dtype: int64

In [495]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday
df.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [496]:
loc = df.index.get_loc('2010-01-05')
loc

print(loc)

df.iloc[loc]["date_time"].strftime("%Y-%m-%d")

1


'2010-01-05'

In [497]:
benzinga_csv_path = "/app/StockPricePredictions/data/benzinga/apple_finbert_20100101_20220304.csv"

In [498]:
df_benzinga = pd.read_csv(benzinga_csv_path, low_memory=False)

In [499]:
df_benzinga['date'] = pd.to_datetime(df_benzinga['Stock'])

In [500]:
df_benzinga["day_of_week"] = df_benzinga["date"].dt.dayofweek

In [501]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     29077 non-null  object        
 1   Stock        29077 non-null  object        
 2   Positive     29077 non-null  float64       
 3   Negative     29077 non-null  float64       
 4   Neutral      29077 non-null  float64       
 5   date         29077 non-null  datetime64[ns]
 6   day_of_week  29077 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 1.6+ MB


In [502]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Time to Sell Apple Puts In case you missed the meteoric run in Apple over the last several months, there may still be a way to profit without having to chase the stock at these lofty levels: selling puts. When you sell puts you in a sense become an insurance salesman. You agree to buy the stock at some point in the future, should the stock fall to that level or lower before a given date. For this agreement, you receive a premium up front. With Apple currently trading at $210/share, just off its 52-week high, I propose selling out-of-the-money puts for April or July. The April $190 contract bids at $8.15 and the July $180 contract bids at $10.25. I chose such long-dated contracts for their healty premiums. Just a reminder that the first rule of thumb with selling puts is to only sell puts on a stock you would be willing to own. Since I firmly believe in Apple's fundamentals and future business prospects, I would willingly buy the stock at $190 in April. However, nothing is certain, and Apple shares currently have a lot of positive news already ""baked in"". If the upcoming tablet is a flop or never arrives at all, or if the company somehow misses earnings forecasts, the stock price may fall sharply. Since I agree with most analysts that Apple will earn roughly $11-12 in non-GAAP EPS in FY2010 and that a $240-260 price target is reasonable, selling out-of-the-money puts appears to be a great way to generate income and set the price you would be willing to pay for Apple shares. Therefore, it is my bet that Apple will stay above $190 by April and $180 by July and the put options will expire worthless, giving the seller a nice premium for either contract.",2010-01-02,0.93772,0.061807,0.000473,2010-01-02,5


In [503]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Headline     29077 non-null  object        
 1   Stock        29077 non-null  object        
 2   Positive     29077 non-null  float64       
 3   Negative     29077 non-null  float64       
 4   Neutral      29077 non-null  float64       
 5   date         29077 non-null  datetime64[ns]
 6   day_of_week  29077 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(1), object(2)
memory usage: 1.6+ MB


In [504]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

df_benzinga.day_of_week.value_counts()

1    6598
2    5977
0    5639
3    5184
4    4465
6    667 
5    547 
Name: day_of_week, dtype: int64

In [505]:
df_benzinga.head(1)

,Headline,Stock,Positive,Negative,Neutral,date,day_of_week
0,"Time to Sell Apple Puts In case you missed the meteoric run in Apple over the last several months, there may still be a way to profit without having to chase the stock at these lofty levels: selling puts. When you sell puts you in a sense become an insurance salesman. You agree to buy the stock at some point in the future, should the stock fall to that level or lower before a given date. For this agreement, you receive a premium up front. With Apple currently trading at $210/share, just off its 52-week high, I propose selling out-of-the-money puts for April or July. The April $190 contract bids at $8.15 and the July $180 contract bids at $10.25. I chose such long-dated contracts for their healty premiums. Just a reminder that the first rule of thumb with selling puts is to only sell puts on a stock you would be willing to own. Since I firmly believe in Apple's fundamentals and future business prospects, I would willingly buy the stock at $190 in April. However, nothing is certain, and Apple shares currently have a lot of positive news already ""baked in"". If the upcoming tablet is a flop or never arrives at all, or if the company somehow misses earnings forecasts, the stock price may fall sharply. Since I agree with most analysts that Apple will earn roughly $11-12 in non-GAAP EPS in FY2010 and that a $240-260 price target is reasonable, selling out-of-the-money puts appears to be a great way to generate income and set the price you would be willing to pay for Apple shares. Therefore, it is my bet that Apple will stay above $190 by April and $180 by July and the put options will expire worthless, giving the seller a nice premium for either contract.",2010-01-02,0.93772,0.061807,0.000473,2010-01-02,5


In [506]:
df_benzinga["in_index"] = df_benzinga["Stock"].apply(lambda x: True if x in df.index else False)

In [507]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

6    667
5    547
0    109
4    22 
3    13 
1    10 
2    4  
Name: day_of_week, dtype: int64

In [508]:
df.index.get_loc('2010-01-04')


0

In [509]:
from datetime import date, datetime, timedelta

In [510]:
test2 =  '2010-01-09'

dtobj1 = datetime.strptime(test2, "%Y-%m-%d")

print(dtobj1)

days = timedelta(2)

print(days)



dtobj2 = (dtobj1 - days).strftime("%Y-%m-%d")
dtobj2

2010-01-09 00:00:00
2 days, 0:00:00


'2010-01-07'

In [511]:
def update_date(x):

    global ERROR_COUNTER

    if x.in_index == True:
        return x.Stock
    else:
        # try:

            dt_time = datetime.strptime(x.Stock, "%Y-%m-%d")

            # loc = df.index.get_loc(date_part)
            if x.day_of_week == 6: # Sunday to Thursday
                days = timedelta(3)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e1:
                    ERROR_COUNTER += 1
                    print(str(e1))
                    return x.Stock
            if x.day_of_week == 5: # Saturday to Thursday
                days = timedelta(2)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e2:
                    ERROR_COUNTER += 1
                    print(str(e2))
                    return x.Stock
            if x.day_of_week == 4: # Friday to Thursday
                days = timedelta(1)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock
            if x.day_of_week == 0: # Monday to Thursday
                days = timedelta(4)
                idx_lookup = (dt_time - days).strftime("%Y-%m-%d")
                print(idx_lookup)
                try:
                    loc = df.index.get_loc(idx_lookup)
                    return df.iloc[loc]["date_time"].strftime("%Y-%m-%d")
                except Exception as e3:
                    ERROR_COUNTER += 1
                    print(str(3))
                    return x.Stock

            else:
                return x.Stock

        # except Exception as e:
        #     ERROR_COUNTER += 1
        #     print(str(e))
        #     print(x.day_of_week, x.Stock, x.in_index)
        #     return x.Stock
    # loc = df.index.get_loc('2010-01-01')
    # loc



In [512]:
# The day of the week with Monday=0, Sunday=6.

# 5 = Saturday
# 6 = Sunday

ERROR_COUNTER = 0 

df_benzinga["backfill_date"] = df_benzinga.apply(update_date, axis=1)


2009-12-31
'2009-12-31'
2010-01-07
2010-01-07
2010-01-14
2010-01-14
2010-01-14
2010-01-14
2010-01-14
2010-01-28
2010-01-28
2010-02-11
2010-02-11
2010-02-11
2010-02-11
2010-02-11
2010-02-11
2010-02-11
2010-02-11
2010-02-11
2010-02-18
2010-02-25
2010-03-04
2010-03-04
2010-03-04
2010-03-11
2010-03-25
2010-03-25
2010-01-28
2010-04-01
2010-04-01
2010-04-15
2010-04-29
2010-05-06
2010-05-06
2010-05-27
2010-05-27
2010-06-03
2010-06-24
2010-06-24
2010-07-01
2010-07-08
2010-07-15
2010-07-15
2010-07-15
2010-07-22
2010-07-22
2010-07-22
2010-07-22
2010-08-05
2010-08-12
2010-08-26
2010-08-26
2010-09-02
2010-09-02
2010-09-02
2010-09-30
2010-10-14
2010-10-14
2010-10-07
2010-11-04
2010-11-18
2010-11-18
2010-11-18
2010-12-02
2010-12-02
2010-12-09
2010-12-16
2010-12-16
2010-12-16
2010-12-23
2011-01-06
2011-01-06
2011-01-13
2011-01-13
2011-01-13
2011-01-13
2011-01-13
2011-01-13
2011-01-13
2011-01-13
2011-01-13
2011-01-20
2011-01-20
2011-01-20
2011-01-27
2011-01-20
2011-02-03
2011-02-10
2011-01-27
2011-02-

In [513]:
ERROR_COUNTER

26

In [514]:
df_benzinga["in_index"] = df_benzinga["backfill_date"].apply(lambda x: True if x in df.index else False)

In [515]:
df_benzinga[df_benzinga.in_index==False]["day_of_week"].value_counts()

5    20
3    13
1    10
6    6 
2    4 
Name: day_of_week, dtype: int64

In [516]:
df_benzinga[df_benzinga.in_index==False].shape

(53, 9)

In [517]:
# df_benzinga["text"] = df_benzinga["title"] + " " + df_benzinga["body"]

In [518]:
# df_benzinga['text'] = np.where(df_benzinga["text"], df_benzinga["title"], df_benzinga["text"])

In [519]:
df_benzinga = df_benzinga[["backfill_date", "Positive", "Negative", "Neutral"]]

In [520]:
df_benzinga.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29077 entries, 0 to 29076
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   backfill_date  29077 non-null  object 
 1   Positive       29077 non-null  float64
 2   Negative       29077 non-null  float64
 3   Neutral        29077 non-null  float64
dtypes: float64(3), object(1)
memory usage: 908.8+ KB


In [521]:
df_benzinga.head()

,backfill_date,Positive,Negative,Neutral
0,2010-01-02,0.937720,0.061807,0.000473
1,2010-01-04,0.010036,0.000452,0.989511
2,2010-01-04,0.006489,0.993502,0.000009
3,2010-01-04,0.036153,0.152172,0.811675
4,2010-01-04,0.961501,0.024860,0.013639


In [522]:
df_benzinga.rename(columns={"backfill_date": "date"}, inplace=True)

In [523]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['text'] = df['text'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [524]:
# df_benzinga = clean_tweet(df_benzinga)

In [525]:
df_benzinga.set_index("date", inplace=True)

In [526]:
df_benzinga.head()

,Positive,Negative,Neutral
date,,,
2010-01-02,0.937720,0.061807,0.000473
2010-01-04,0.010036,0.000452,0.989511
2010-01-04,0.006489,0.993502,0.000009
2010-01-04,0.036153,0.152172,0.811675
2010-01-04,0.961501,0.024860,0.013639


In [527]:
df_benzinga_duplicated_index = df_benzinga[df_benzinga.index.duplicated(keep=False)]

In [528]:
df_benzinga_duplicated_index.shape[0]

29014

In [529]:
df_benzinga_nonduplicated_index = df_benzinga[~df_benzinga.index.duplicated(keep=False)]

In [530]:
df_benzinga_nonduplicated_index.shape[0]

63

In [531]:
# df_twint.groupby('date')['negative'].mean()

df_benzinga_groupby = df_benzinga.groupby("date").agg(
     negative = ("Negative", "mean"),
     nuetral = ("Neutral", "mean"),
     positive = ("Positive", "mean"),
     # compound = ("compound", "mean"),
     # nlikes = ("nlikes", "sum"),
     # nretweets = ("nretweets", "sum"),
     )


In [532]:
df_benzinga_groupby

,negative,nuetral,positive
date,,,
2010-01-02,0.061807,0.000473,0.937720
2010-01-04,0.378122,0.190962,0.430915
2010-01-05,0.487177,0.211328,0.301494
2010-01-06,0.479227,0.066754,0.454019
2010-01-07,0.209091,0.169873,0.621037
...,...,...,...
2022-02-28,0.352046,0.020751,0.627203
2022-03-01,0.236412,0.372266,0.391322
2022-03-02,0.168162,0.071168,0.760670


In [533]:
df_merge = pd.merge(df, df_benzinga_groupby, how="left", left_index=True, right_index=True)

In [534]:
df_merge.shape

(3064, 16)

In [535]:
df_merge.day_of_week.value_counts()

1    629
2    628
3    619
4    614
0    574
Name: day_of_week, dtype: int64

In [536]:
# df.index
# df_text.index
df_merge[df_merge.negative.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2012-08-10,618.71,621.76,618.7000,621.70,19.091146,6962100.0,0.00,1.0,2012-08-10,4,19.346023,1.335049,1,NaN,NaN,NaN
2016-06-22,96.25,96.89,95.3500,95.55,22.217966,29219122.0,0.00,1.0,2016-06-22,2,22.345856,0.575615,1,NaN,NaN,NaN
2016-11-04,108.53,110.25,108.1100,108.84,25.576562,30836997.0,0.00,1.0,2016-11-04,4,25.945500,1.442484,1,NaN,NaN,NaN
2016-12-23,115.59,116.52,115.5900,116.52,27.381303,14249484.0,0.00,1.0,2016-12-23,4,27.555198,0.635084,1,NaN,NaN,NaN
2017-11-24,175.10,175.50,174.6459,174.97,41.780905,14026519.0,0.00,1.0,2017-11-24,4,41.570770,-0.502943,0,NaN,NaN,NaN
2018-02-23,173.67,175.65,173.5400,175.50,42.076261,33329232.0,0.00,1.0,2018-02-23,4,42.908196,1.977208,1,NaN,NaN,NaN
2018-03-02,172.80,176.30,172.4500,176.21,42.246484,38453950.0,0.00,1.0,2018-03-02,4,42.392732,0.346178,1,NaN,NaN,NaN
2018-03-09,177.96,180.00,177.3900,179.98,43.150344,31385134.0,0.00,1.0,2018-03-09,4,43.567511,0.966774,1,NaN,NaN,NaN
2018-07-09,189.50,190.68,189.3000,190.58,45.868567,19756634.0,0.00,1.0,2018-07-09,0,45.813211,-0.120684,0,NaN,NaN,NaN


In [537]:
df.shape

(3064, 13)

In [538]:
df_benzinga_groupby.shape

(3073, 3)

In [539]:
df_merge.fillna(method="ffill", inplace=True)

In [540]:
df_merge[df_merge.negative.isnull()]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,


In [541]:
# df_merge.to_csv("/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL_WITH_BENZINGA_FINBERT_SA.csv")
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,date_time,day_of_week,adjusted_close_shift,percentage_change,increase,negative,nuetral,positive
date,,,,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,2010-01-04,0,6.555190,0.172889,1,0.378122,0.190962,0.430915
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,2010-01-05,1,6.450921,-1.590633,0,0.487177,0.211328,0.301494
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,2010-01-06,2,6.438996,-0.184860,0,0.479227,0.066754,0.454019
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,2010-01-07,3,6.481804,0.664830,1,0.209091,0.169873,0.621037
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,2010-01-08,4,6.424624,-0.882159,0,0.421060,0.149651,0.429289


In [542]:
# df_merge = df_merge.sample(frac=1).reset_index(drop=True)

In [543]:
X = df_merge[["negative", "nuetral", "positive"]]
y = df_merge["increase"]

In [544]:
y.head()

date
2010-01-04    1
2010-01-05    0
2010-01-06    0
2010-01-07    1
2010-01-08    0
Name: increase, dtype: int64

In [545]:
SPLIT = int(0.9 * len(df_merge))

X_train = X[:SPLIT]
X_test = X[SPLIT:]

y_train = y[:SPLIT]
y_test = y[SPLIT:]



In [546]:
X_train

,negative,nuetral,positive
date,,,
2010-01-04,0.378122,0.190962,0.430915
2010-01-05,0.487177,0.211328,0.301494
2010-01-06,0.479227,0.066754,0.454019
2010-01-07,0.209091,0.169873,0.621037
2010-01-08,0.421060,0.149651,0.429289
...,...,...,...
2020-12-08,0.261165,0.122691,0.616144
2020-12-09,0.200314,0.015639,0.784047
2020-12-10,0.111745,0.123594,0.764661


In [547]:
from sklearn.ensemble import RandomForestClassifier

In [548]:
rfc = RandomForestClassifier(n_estimators=200, criterion="entropy")
rfc.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [549]:
preds = rfc.predict(X_test)

In [550]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [551]:
matrix = confusion_matrix(y_test, preds)

In [552]:
matrix

array([[72, 79],
       [69, 87]])

In [553]:
score = accuracy_score(y_test, preds)

In [554]:
score

0.5179153094462541

In [555]:
print(classification_report(y_test, preds))


              precision    recall  f1-score   support

           0       0.51      0.48      0.49       151
           1       0.52      0.56      0.54       156

    accuracy                           0.52       307
   macro avg       0.52      0.52      0.52       307
weighted avg       0.52      0.52      0.52       307



In [556]:
corr_matrix = df_merge[["negative", "nuetral", "positive", "increase"]].corr()
print (corr_matrix)

          negative   nuetral  positive  increase
negative  1.000000 -0.225683 -0.743840 -0.004286
nuetral  -0.225683  1.000000 -0.483243 -0.021125
positive -0.743840 -0.483243  1.000000  0.018344
increase -0.004286 -0.021125  0.018344  1.000000


In [558]:
from statsmodels.tsa.stattools import grangercausalitytests

In [563]:
#perform Granger-Causality test
grangercausalitytests(df_merge[["positive", "increase"]], maxlag=[1])


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0945  , p=0.7586  , df_denom=3060, df_num=1
ssr based chi2 test:   chi2=0.0946  , p=0.7584  , df=1
likelihood ratio test: chi2=0.0946  , p=0.7584  , df=1
parameter F test:         F=0.0945  , p=0.7586  , df_denom=3060, df_num=1


{1: ({'ssr_ftest': (0.09448947774012272, 0.7585663854392968, 3060.0, 1),
   'ssr_chi2test': (0.09458211448300517, 0.7584308190747939, 1),
   'lrtest': (0.0945806542167702, 0.7584326258377987, 1),
   'params_ftest': (0.09448947773913842, 0.758566385440138, 3060.0, 1.0)},
   array([[0., 1., 0.]])])}